### Read the JSON file 

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/utils"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [0]:
pit_stop_schema = StructType([StructField("raceId", IntegerType(), False), 
                              StructField("driverId", IntegerType(), True),
                              StructField("stop", StringType(), True),
                              StructField("lap", IntegerType(), True),
                              StructField("time", StringType(), True),
                              StructField("duration", StringType(), True),
                              StructField("milliseconds", IntegerType(), True),
                            ])


pit_stops_df = spark.read \
    .schema(pit_stop_schema) \
    .option("multiline", True) \
    .json(f"{RAW_FOLDER_PATH}/pit_stops.json")   


In [0]:
pit_stops_df.printSchema()

In [0]:
display(pit_stops_df)

### rename columns and add new column

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, date_trunc, from_utc_timestamp

In [0]:
final_pit_stops_df = add_ingestion_date(pit_stops_df.withColumnRenamed("driverId", "driver_id") \
                                .withColumnRenamed("raceId", "race_id")) 



display(final_pit_stops_df)

### write to parquet file

In [0]:
final_pit_stops_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.pit_stops")



In [0]:
spark.read.parquet(f"{PROCESSED_FOLDER_PATH}/pit_stops").display()

In [0]:
dbutils.notebook.exit("Success")